In [7]:
import uuid
import yaml
import time
from julep import Client

TRACKER_UUID = uuid.uuid4()
BUDGET_TASK_UUID = uuid.uuid4()
ADVICE_TASK_UUID = uuid.uuid4()

api_key = ""  # Your API key here
client = Client(api_key=api_key, environment="dev")

agent = client.agents.create_or_update(
    agent_id=TRACKER_UUID,
    name="Personal Finance Tracker",
    about="Tracks user expenses, analyzes spending patterns, and provides financial advice.",
    model="gpt-4o",
)

log_expense_task_def = yaml.safe_load(f"""
name: Log Expense

input_schema:
  type: object
  properties:
    amount:
      type: number
    category:
      type: string
    description:
      type: string

tools:
- name: finance_tool
  system:
    resource: agent
    subresource: tool
    operation: create

main:
- tool: finance_tool
  arguments:
    agent_id: "'{agent.id}'"
    data:
      amount: inputs[0].amount
      category: inputs[0].category
      description: inputs[0].description
""")

log_expense_task = client.tasks.create_or_update(
    task_id=TRACKER_UUID,
    agent_id=TRACKER_UUID,
    **log_expense_task_def
)

set_budget_task_def = yaml.safe_load(f"""
name: Set Budget

input_schema:
  type: object
  properties:
    category:
      type: string
    budget_amount:
      type: number

tools:
- name: finance_tool
  system:
    resource: agent
    subresource: tool
    operation: create

main:
- tool: finance_tool
  arguments:
    agent_id: "'{agent.id}'"
    data:
      category: inputs[0].category
      budget_amount: inputs[0].budget_amount
""")

set_budget_task = client.tasks.create_or_update(
    task_id=BUDGET_TASK_UUID,
    agent_id=TRACKER_UUID,
    **set_budget_task_def
)

send_advice_task_def = yaml.safe_load(f"""
name: Send Financial Advice

input_schema:
  type: object
  properties:
    advice:
      type: string
    email:
      type: string

tools:
- name: email_tool
  system:
    resource: agent
    subresource: tool
    operation: create

main:
- tool: email_tool
  arguments:
    agent_id: "'{agent.id}'"
    to: inputs[0].email
    subject: "Financial Advice"
    body: inputs[0].advice
""")

send_advice_task = client.tasks.create_or_update(
    task_id=ADVICE_TASK_UUID,
    agent_id=TRACKER_UUID,
    **send_advice_task_def
)

def log_expense(amount, category, description):
    execution = client.executions.create(
        task_id=TRACKER_UUID,
        input={"amount": amount, "category": category, "description": description}
    )
    print("Logging expense...")
    time.sleep(5)
    result = client.executions.get(execution.id)
    print(f"Expense logged: {description} of ${amount} in {category} category.")

def set_budget(category, budget_amount):
    execution = client.executions.create(
        task_id=BUDGET_TASK_UUID,
        input={"category": category, "budget_amount": budget_amount}
    )
    print("Setting budget...")
    time.sleep(5)
    result = client.executions.get(execution.id)
    print(f"Budget set for {category}: ${budget_amount}.")

def send_financial_advice(email, advice):
    execution = client.executions.create(
        task_id=ADVICE_TASK_UUID,
        input={"advice": advice, "email": email}
    )
    print("Sending financial advice...")
    time.sleep(5)
    result = client.executions.get(execution.id)
    print(f"Advice sent to {email}: '{advice}'.")

log_expense(50, "Groceries", "Weekly grocery shopping")
set_budget("Groceries", 200)
send_financial_advice("user@example.com", "Consider reducing your grocery budget to save more.")


Logging expense...
Expense logged: Weekly grocery shopping of $50 in Groceries category.
Setting budget...
Budget set for Groceries: $200.
Sending financial advice...
Advice sent to user@example.com: 'Consider reducing your grocery budget to save more.'.
